# Instruction

**Run each cell one by one**

# Load data


In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install mne



import numpy as np
import scipy.io as sio
import mne
def indexget(total,select):
  return [i for i, e in enumerate(total) if e in select]



def inputmat(fp1,fp2,fp3):
    """load .mat file and return m as a dict"""
    mat = sio.loadmat(fp1, squeeze_me=True)
    label_mat=sio.loadmat(fp2, squeeze_me=True)
    m = {}  # create a dict

    # Numpy array of size channel_num * points.
    select_channel=fp3
    channelindex= indexget(mat['nfo']['clab'][True][0],select_channel)
    m['ch_names'] = mat['nfo']['clab'][True][0][channelindex]
    


    m['data'] = mat['cnt'].T[channelindex]
    m['freq'] = mat['nfo']['fs'][True][0]  # Sampling frequency

    # channel names are necessary information for creating a rawArray.


    # Position of channels
    m['electrode_x'] = mat['nfo']['xpos'][True][0]
    m['electrode_y'] = mat['nfo']['ypos'][True][0]

    # find trials and its data
    m['cue'] = mat['mrk']['pos'][True][0]  # time of cue
    m['labels'] = label_mat['true_y']  
    # m['labels'] = np.nan_to_num(mat['mrk']['y'][True][0]).astype(int)  # convert NaN to 0
    m['test_idx'] = len(label_mat['test_idx'])
    m['n_trials'] = len(m['labels'])  # Number of the total useful trials
    return m


def creatEventsArray(fp1,fp2,fp3):
    """Create events array. The second column default to zero."""
    m = inputmat(fp1,fp2,fp3)
    events = np.zeros((m['n_trials'], 3), int)
    events[:, 0] = m['cue'][:m['n_trials']]  # The first column is the sample number of the event.
    events[:, 2] = m['labels'][:m['n_trials']]  # The third column is the new event value.
    return events, m['labels']


def creatRawArray(fp1,fp2,fp3):
    """Create a mne.io.RawArray object, data: array, shape (n_channels, n_times)"""
    m = inputmat(fp1,fp2,fp3)
    ch_names = m['ch_names'].tolist()
    info = mne.create_info(ch_names, m['freq'], 'eeg')  # Create info for raw
    raw = mne.io.RawArray(m['data'], info, first_samp=0, copy='auto', verbose=None)
    return raw

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 6.9 MB/s 


# Package

In [2]:
import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.function import Function
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
from sklearn.decomposition import PCA
import torch.utils.data as Data
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary
import torch.optim.lr_scheduler as lr_scheduler


# Load data

In [3]:
print(__doc__)

fp = {
    'aa': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_aa.mat',
    'al': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_al.mat',
    'av': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_av.mat',
    'aw': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_aw.mat',
    'ay': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_ay.mat',
}


fplabel={
    'aa': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_aa.mat',
    'al': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_al.mat',
    'av': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_av.mat',
    'aw': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_aw.mat',
    'ay': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_ay.mat',

}

channel_set=['C1',  'C3',  'Cz',  'C2',  'C4',  'CFC3',  'CFC4','CFC5',  'CFC6',  'CCP3', 
             'CCP4',  'CCP5',  'CCP6',  'T7',  'T8',  'P3',  'Pz',  'P4']
pick_chan = {
    'aa':  channel_set,
    'al':  channel_set,
    'av':  channel_set,
    'aw':  channel_set,
    'ay':  channel_set,

}

low_freq, high_freq = 7., 30.
tmin, tmax = 0., 3.5
# event_id
event_id = {'right': 1, 'foot': 2}

Automatically created module for IPython interactive environment


In [4]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

from mne.decoding import CSP
from mne.channels import read_layout



def cal_acc(pred,real):
  return [1 if pred[j]==real[j] else 0 for j in range(len(real))]

def getdata_label(a,b,c):
  mat_dic=inputmat(a,b,c)
  raw = creatRawArray(a,b,c)
  events, labels = creatEventsArray(a,b,c)


    # Apply band-pass filter
  raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')

    # event_train = eventsTrain(fp[f])
  epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)

  epochs_train = epochs.copy().crop(tmin=0.5, tmax=2.5)
  labels = epochs.events[:, -1]

    # # Define a monte-carlo cross-validation generator (reduce variance):
  epochs_data= epochs_train.get_data()
  return epochs_data,labels,mat_dic['test_idx']

intial_dic={}

total_length=280
train_num=10
test_num=10

for f,fla,cs in zip(fp,fplabel,pick_chan):
    x,y,testix=getdata_label(fp[f],fplabel[fla],pick_chan[cs])

    intial_dic[f]=[x,y]
    # # Assemble a classifier
    # 
    # csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)





Creating RawArray with float64 data, n_channels=18, n_times=298458
    Range : 0 ... 298457 =      0.000 ...  2984.570 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 165 samples (1.650 sec)

Creating RawArray with float64 data, n_channels=18, n_times=283574
    Range : 0 ... 283573 =      0.000 ...  2835.730 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal 

# Network


In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch

hyperparameter_se=10


class EEGNet(nn.Module):
    def __init__(self, classes_num):
        super(EEGNet, self).__init__()
        self.numC=classes_num

        self.drop_out = 0
        
        self.block_1 = nn.Sequential(
            # Pads the input tensor boundaries with zero
            # left, right, up, bottom
            nn.ZeroPad2d((15, 16, 0, 0)),
            nn.Conv2d(
                in_channels=1,          # input shape (1, C, T)
                out_channels=8,         # num_filters
                kernel_size=(1, 32),    # filter size
                bias=False
            ),                          # output shape (8, C, T)
            nn.BatchNorm2d(8)           # output shape (8, C, T)
        )
        
        # block 2 and 3 are implementations of Depthwise Convolution and Separable Convolution
        self.block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=8,          # input shape (8, C, T)
                out_channels=16,        # num_filters
                kernel_size=(self.numC, 1),    # filter size
                groups=8,
                bias=False
            ),                          # output shape (16, 1, T)
            nn.BatchNorm2d(16),         # output shape (16, 1, T)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//4)
            nn.Dropout(self.drop_out)   # output shape (16, 1, T//4)
        )
        
        self.block_3 = nn.Sequential(
            nn.ZeroPad2d((7, 8, 0, 0)),
            nn.Conv2d(
               in_channels=16,          # input shape (16, 1, T//4)
               out_channels=16,         # num_filters
               kernel_size=(1, 16),     # filter size
               groups=16,
               bias=False
            ),                          # output shape (16, 1, T//4)
            nn.Conv2d(
                in_channels=16,         # input shape (16, 1, T//4)
                out_channels=16,        # num_filters
                kernel_size=(1, 1),     # filter size
                bias=False
            ),                          # output shape (16, 1, T//4)
            nn.BatchNorm2d(16),         # output shape (16, 1, T//4)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//32)
            nn.Dropout(self.drop_out)
        )

    
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        return x 


    def get_embedding(self, x):
        return self.forward(x)





class RelationNetwork(nn.Module):
    """docstring for RelationNetwork"""
    def __init__(self,hidden_size):
        super(RelationNetwork, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(32,16,kernel_size=(1,2),padding=1),
                        nn.BatchNorm2d(16, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(16,16,kernel_size=(1,2),padding=1),
                        nn.BatchNorm2d(16, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.fc1 = nn.Linear(32,hidden_size)
        self.fc2 = nn.Linear(hidden_size,1)

    def forward(self,x):
        x=x.view(-1,32,1,3)
        out = self.layer1(x)
        # out = self.layer2(out)
        out = out.view(out.size(0),-1)
        
        out = F.relu(self.fc1(out))
        out = torch.sigmoid(self.fc2(out))
        return out

# Data generator

In [6]:
def mini_set(label_set,num):
  index1=np.where(label_set == 0)[0]
  index2=np.where(label_set == 1)[0]
  number=max(index1[num-1],index2[num-1])+1
  return number
  


In [7]:
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch
from torch.utils.data import DataLoader,Dataset
import random
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.sampler import Sampler

class BCI_task(object):

    def __init__(self, feature_list,label,train_num,test_num,settype):

        self.train_num = train_num
        self.test_num = test_num
        if settype=="test":
          testmode=mini_set(label,train_num)
          # testmode=40
          supportset=label[:testmode]
          index1=np.where(supportset == 0)[0]
          index2=np.where(supportset == 1)[0]
          choice1=np.random.choice(index1,train_num,replace=False)
          choice2=np.random.choice(index2,train_num,replace=False)
          choice_total=np.hstack((choice1,choice2))
          self.training_feature = feature_list[choice_total]
          self.train_labels= label[choice_total]
          test_list=list(range(testmode))

          self.testing_feature= np.delete(feature_list, test_list,0)
          self.test_labels=np.delete(label, test_list)

        else:
          index1=np.where(label == 0)[0]
          index2=np.where(label == 1)[0]
          choice1=np.random.choice(index1,train_num+test_num,replace=False)
          choice2=np.random.choice(index2,train_num+test_num,replace=False)
          train_choice=np.hstack((choice1[:train_num],choice2[:train_num]))
          test_choice=np.hstack((choice1[train_num:],choice2[train_num:]))

    

          self.training_feature = feature_list[train_choice]
          self.train_labels= label[train_choice]



          self.testing_feature = feature_list[test_choice]
          self.test_labels =label[test_choice]



class FewShotDataset(Dataset):

    def __init__(self, task, split='train'):
        self.task = task
        self.split = split
        self.feature = self.task.training_feature if self.split == 'train' else self.task.testing_feature
        self.labels = self.task.train_labels if self.split == 'train' else self.task.test_labels

    def __len__(self):
        return len(self.feature)

    def __getitem__(self, idx):
        raise NotImplementedError("This is an abstract class. Subclass this class for your particular dataset.")


class BCI_DATA(FewShotDataset):
    def __init__(self, *args, **kwargs):
        super(BCI_DATA, self).__init__(*args, **kwargs)

    def __getitem__(self, idx):
        feature = self.feature[idx]
        label = self.labels[idx]
        return feature.astype(np.float32), label


def get_data_loader(task, split='train',shuffle=False):
    dataset = BCI_DATA(task,split=split)
    number = len(task.training_feature) if split == 'train' else len(task.testing_feature)
    loader = DataLoader(dataset, batch_size=number,shuffle=shuffle)
    return loader

# Weight initializer

In [8]:
import math

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('BatchNorm') != -1:
        m.weight.data.fill_(1)
        m.bias.data.zero_()
    elif classname.find('Linear') != -1:
        n = m.weight.size(1)
        m.weight.data.normal_(0, 0.01)
        m.bias.data = torch.ones(m.bias.data.size())

# Hyperparameter

In [9]:
TRAIN_NUM=hyperparameter_se
TEST_NUM=hyperparameter_se
LEARNING_RATE=0.001

EPSIODE_NUM=2000

FEATURE_DIM=240
RELATION_DIM=240
n_layers=1
MIDDLE_DIM=240

#Main

In [10]:
dic=[]
str_list=['aa','al','av','aw','ay']


for i in str_list:
  temp=[]
  x,y=intial_dic[i]
  x=x.reshape(x.shape[0],1,x.shape[1],-1)

  y=y-1
  temp.append(x)
  temp.append(y)

  dic.append(temp)



In [23]:
def main(Subject_number):


  GPU=0
  feature_encoder = EEGNet(classes_num=18)
  relation_network = RelationNetwork(FEATURE_DIM)

  feature_encoder.apply(weights_init)
  relation_network.apply(weights_init)
  GPU=0

  feature_encoder.cuda(GPU)
  relation_network.cuda(GPU)



  x_test=dic[Subject_number][0]
  y_test=dic[Subject_number][1]


  full_subjectnum=[0,1,2,3,4]
  full_subjectnum.remove(Subject_number)

  feature_encoder_optim = torch.optim.Adam(feature_encoder.parameters(),lr=LEARNING_RATE)
  relation_network_optim = torch.optim.Adam(relation_network.parameters(),lr=LEARNING_RATE)

  total_losss=[]
  testlossset=[]
  for episode in range(EPSIODE_NUM):
    select_subject=np.random.choice(full_subjectnum, 1)[0]
    x_train,y_train = dic[select_subject][0],dic[select_subject][1]


    bci_task=BCI_task(x_train,y_train,train_num=TRAIN_NUM,test_num=TEST_NUM,settype='train')
    sample_dataloader = get_data_loader(bci_task,split="train",shuffle=False)
    batch_dataloader = get_data_loader(bci_task,split="test",shuffle=False)

    samples,sample_labels = sample_dataloader.__iter__().next()
    batches,batch_labels = batch_dataloader.__iter__().next()


    ### 
    #Directly calculate each score
    ##
 
    #sum method pending to write
    sample_labels=sample_labels.type(torch.int32).cuda(GPU)
    sample_features = feature_encoder(Variable(samples).cuda(GPU)) # 10x16*1*4
    nrow = torch.unique(sample_labels).size(0)
    out = torch.zeros((nrow, sample_features.size(1),sample_features.size(2),sample_features.size(3)), dtype=sample_features.dtype).cuda(GPU)
    out.index_add_(0, sample_labels, sample_features)
    sample_features=out    # 2x16*1*4
    batch_features = feature_encoder(Variable(batches).cuda(GPU)) # 5x16*1*4




    sample_features_ext = sample_features.unsqueeze(0).repeat(len(bci_task.testing_feature),1,1,1,1)
    batch_features_ext = batch_features.unsqueeze(0).repeat(sample_features.size(0),1,1,1,1)
    batch_features_ext = torch.transpose(batch_features_ext,0,1)


    relation_pairs = torch.cat((sample_features_ext,batch_features_ext),2).view(-1,batch_features_ext.size()[2]*2,batch_features_ext.size()[3],batch_features_ext.size()[4])
    relations = relation_network(relation_pairs)
    relations = relations.view(len(bci_task.testing_feature),-1)
    mse = nn.MSELoss(reduction='mean').cuda(GPU)
    one_hot_labels =F.one_hot(batch_labels.to(torch.int64), num_classes=2).float().cuda(GPU)

    loss = mse(relations,one_hot_labels)

    feature_encoder.zero_grad()
    relation_network.zero_grad()

    loss.backward()


    torch.nn.utils.clip_grad_norm_(feature_encoder.parameters(),0.5)
    torch.nn.utils.clip_grad_norm_(relation_network.parameters(),0.5)

    feature_encoder_optim.step()
    relation_network_optim.step()
    total_losss.append(loss.item())



    if (episode+1)%200 == 0:
        total_losss=[]
        total_rewards = 0
        bci_task=BCI_task(x_test,y_test,train_num=TRAIN_NUM,test_num=TEST_NUM,settype='test')
        sample_dataloader = get_data_loader(bci_task,split="train",shuffle=False)
        batch_dataloader = get_data_loader(bci_task,split="test",shuffle=False)

        samples,sample_labels = sample_dataloader.__iter__().next()
        batches,batch_labels = batch_dataloader.__iter__().next()

        sample_labels=sample_labels.type(torch.int32).cuda(GPU)
        sample_features = feature_encoder(Variable(samples).cuda(GPU))
        nrow = torch.unique(sample_labels).size(0)
        out = torch.zeros((nrow, sample_features.size(1),sample_features.size(2),sample_features.size(3)), dtype=sample_features.dtype).cuda(GPU)
        out.index_add_(0, sample_labels, sample_features)
        sample_features=out    # 2x16*1*4
        batch_features = feature_encoder(Variable(batches).cuda(GPU)) 



        sample_features_ext = sample_features.unsqueeze(0).repeat(len(bci_task.testing_feature),1,1,1,1)
        batch_features_ext = batch_features.unsqueeze(0).repeat(sample_features.size(0),1,1,1,1)
        batch_features_ext = torch.transpose(batch_features_ext,0,1)
        relation_pairs = torch.cat((sample_features_ext,batch_features_ext),2).view(-1,batch_features_ext.size()[2]*2,batch_features_ext.size()[3],batch_features_ext.size()[4])
        relations = relation_network(relation_pairs)
        relations = relations.view(len(bci_task.testing_feature),-1)
        _,predict_labels = torch.max(relations.data,1)
        rewards = [1 if predict_labels[j]==batch_labels[j] else 0 for j in range(len(bci_task.testing_feature))]
        testlossset.append(np.sum(rewards)/len(rewards)*100)



  print("Last episode test accuracy:",np.sum(rewards)/len(rewards)*100,"%")
  print("Highest three each 200 episodes:",np.sort(testlossset)[-3:])




In [24]:
Subject_number=2
main(Subject_number)

Last episode test accuracy: 59.45945945945946 %
Highest three each 200 episodes: [63.70656371 65.25096525 66.7953668 ]
